In [3]:
# Install required packages.
!pip install -q torch-cluster 
!pip install -q torch-scatter 
!pip install -q torch-sparse 
!pip install -q torch-geometric

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.8/53.8 kB 5.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.6/107.6 kB 10.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.2/209.2 kB 11.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 661.6/661.6 kB 37.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
from torch_geometric.nn import Node2Vec
import os.path as osp
import torch
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
from torch_geometric.datasets import Planetoid
from tqdm.notebook import tqdm
import os

PRJ_DIR = os.path.join('/','content','drive','MyDrive','Colab Notebooks','eliorc_node2vec')
DATA_DIR = os.path.join(PRJ_DIR,'Facebook-Social-Network-Analysis-master')

dataset = 'Cora'
path = osp.join('.', 'data', dataset)
dataset = Planetoid(path, dataset)
data = dataset[0]

Processing...
Done!


In [6]:
import pandas as pd

edge_tensor = torch.permute(data.edge_index, (1, 0))
cora_df = pd.DataFrame(edge_tensor, columns = ['Source','Destination'])
print(cora_df.head())

   Source  Destination
0       0          633
1       0         1862
2       0         2582
3       1            2
4       1          652


In [7]:
#@title Creating graph

import networkx as nx

#creating a list of node pair tuples from the 'cora_df' dataframe
existing_edges = [(cora_df.iloc[i,0],cora_df.iloc[i,1]) for i in range(cora_df.shape[0])]

cora_G = nx.from_pandas_edgelist(cora_df, "Source", "Destination", create_using=nx.DiGraph())

#cora_G.add_edges_from(existing_edges)
print("------Before Dropping nodes with low degree (<3)-----")
print('Number of nodes', len(cora_G.nodes))
print('Number of edges', len(cora_G.edges))
print('Average degree', sum(dict(cora_G.degree).values()) / len(cora_G.nodes))

------Before Dropping nodes with low degree (<3)-----
Number of nodes 2708
Number of edges 10556
Average degree 7.796159527326441


In [8]:
print(cora_G.has_edge(2, 1))

True


In [9]:
#@title Creating Adjacency Matrix

nodelist = list(cora_G.nodes())
adj_cora = nx.to_numpy_array(cora_G, nodelist = nodelist)
print(str(adj_cora.shape)+'\n')
print(adj_cora)

(2708, 2708)

[[0. 1. 1. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [10]:
import random

non_existing_edges_20000 = []
while(len(non_existing_edges_20000) < 20000):
    source_idx = random.randint(0, adj_cora.shape[0]-1)
    destination_idx = random.randint(0, adj_cora.shape[1]-1)
    if source_idx == destination_idx or adj_cora[source_idx, destination_idx] == 1:
        continue

    if nx.has_path(cora_G, nodelist[source_idx], nodelist[destination_idx]):
        non_existing_edges_20000.extend([(nodelist[source_idx],nodelist[destination_idx])])
    

non_existing_edge_df = pd.DataFrame(data = non_existing_edges_20000, columns =['Source', 'Destination'])
non_existing_edge_df['Connection'] = 0
print(non_existing_edge_df.head())

   Source  Destination  Connection
0    2099          942           0
1    2374         2588           0
2    2653         1156           0
3    1889         1211           0
4    2284         1062           0


In [11]:
cora_df[:10]

,Source,Destination
0,0,633
1,0,1862
2,0,2582
3,1,2
4,1,652
5,1,654
6,2,1
7,2,332
8,2,1454
9,2,1666


In [12]:
remove_edge_df = cora_df.sample(n=len(cora_df.index)//2, random_state=1)
remove_edge_df['Connection'] = 1
print(remove_edge_df.head())

      Source  Destination  Connection
9496    2309          994           1
2943     734          408           1
783      175         2135           1
391       91         2123           1
1168     290         1765           1


In [13]:
#@title Creating Subgraph
non_existing_edge_df = non_existing_edge_df.append(remove_edge_df[['Source', 'Destination', 'Connection']],
                ignore_index=True)

edges_cora_G_past = cora_df.drop(index=remove_edge_df.index.values)

cora_G_new = nx.from_pandas_edgelist(edges_cora_G_past, "Source", "Destination",
                               create_using=nx.DiGraph())

print('Number of nodes', len(cora_G_new.nodes))
print('Number of edges', len(cora_G_new.edges))
print('Average degree', sum(dict(cora_G_new.degree).values()) / len(cora_G_new.nodes))

Number of nodes 2559
Number of edges 5278
Average degree 4.12504884720594


<ipython-input-13-79d86168b2a7>:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  non_existing_edge_df = non_existing_edge_df.append(remove_edge_df[['Source', 'Destination', 'Connection']],


In [18]:
from torch_geometric.nn import Node2Vec
import os.path as osp
import torch
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
from torch_geometric.datasets import Planetoid
from tqdm.notebook import tqdm

from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.metrics import f1_score, auc, roc_curve, roc_auc_score,confusion_matrix
from sklearn.metrics import accuracy_score, ConfusionMatrixDisplay
import numpy as np

def train(model, loader, optimizer):
    model.train()
    total_loss = 0
    for pos_rw, neg_rw in loader:
        optimizer.zero_grad()
        loss = model.loss(pos_rw.to(device), neg_rw.to(device))
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    return total_loss / len(loader)

device = 'cuda' if torch.cuda.is_available() else 'cpu'

edges_tensor = torch.from_numpy(edges_cora_G_past.to_numpy())
edges_tensor = torch.permute(edges_tensor, (1, 0))

set_of_pq = [0.25, 0.50, 1, 2, 4]

model_pool = {}

for p_value in set_of_pq:
    for q_value in set_of_pq:
        node2vec = Node2Vec(edge_index = edges_tensor,
                    embedding_dim = 128,
                    walk_length = 16,
                    context_size = 10,
                    walks_per_node = 20,
                    p=p_value, q=q_value).to(device)

        loader = node2vec.loader(batch_size=128, shuffle=True, num_workers=4)
        optimizer = torch.optim.Adam(list(node2vec.parameters()), lr=0.01)

        print(f"-------Training with p={p_value} and q={q_value}--------")
        for epoch in range(1, 150):
            loss = train(node2vec, loader, optimizer)
            if epoch % 15 == 0:
                print(f'Epoch: {epoch:02d}, Loss: {loss:.4f}')

        embedding_nodes = node2vec(torch.arange(node2vec.embedding.weight.size()[0], device=device))

        edge_features = [
            (embedding_nodes[i].detach().cpu().numpy() + embedding_nodes[j].detach().cpu().numpy()) 
            for i,j in zip(non_existing_edge_df['Source'], non_existing_edge_df['Destination'])
        ]

        X = np.array(edge_features)  
        y = non_existing_edge_df['Connection']

        X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = random.randint(0, 999999))

        #classifier
        random_forest_clf = RandomForestClassifier(n_estimators=100, max_depth=15)
        
        # alternative metric to optimize over grid parameters: AUC
        random_forest_clf.fit(X_train, y_train)
        
        model_pool[f"model_pq:{p_value}_{q_value}"] = random_forest_clf
        model_pool[f"X_test_pq:{p_value}_{q_value}"] = X_test
        model_pool[f"y_test_pq:{p_value}_{q_value}"] = y_test

-------Training with p=0.25 and q=0.25--------


/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:561: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


Epoch: 15, Loss: 1.0771
Epoch: 30, Loss: 0.8102
Epoch: 45, Loss: 0.7756
Epoch: 60, Loss: 0.7654
Epoch: 75, Loss: 0.7607
Epoch: 90, Loss: 0.7599
Epoch: 105, Loss: 0.7585
Epoch: 120, Loss: 0.7574
Epoch: 135, Loss: 0.7576
-------Training with p=0.25 and q=0.5--------


/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:561: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


Epoch: 15, Loss: 1.1024
Epoch: 30, Loss: 0.8145
Epoch: 45, Loss: 0.7751
Epoch: 60, Loss: 0.7633
Epoch: 75, Loss: 0.7575
Epoch: 90, Loss: 0.7558
Epoch: 105, Loss: 0.7543
Epoch: 120, Loss: 0.7535
Epoch: 135, Loss: 0.7523
-------Training with p=0.25 and q=1--------


...

Epoch: 15, Loss: 1.0812
Epoch: 30, Loss: 0.8118
Epoch: 45, Loss: 0.7766
Epoch: 60, Loss: 0.7665
Epoch: 75, Loss: 0.7620
Epoch: 90, Loss: 0.7602
Epoch: 105, Loss: 0.7591
Epoch: 120, Loss: 0.7580
Epoch: 135, Loss: 0.7583
-------Training with p=4 and q=2--------


/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:561: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


Epoch: 15, Loss: 1.0814
Epoch: 30, Loss: 0.8099
Epoch: 45, Loss: 0.7756
Epoch: 60, Loss: 0.7650
Epoch: 75, Loss: 0.7609
Epoch: 90, Loss: 0.7593
Epoch: 105, Loss: 0.7582
Epoch: 120, Loss: 0.7576
Epoch: 135, Loss: 0.7568
-------Training with p=4 and q=4--------


/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:561: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


Epoch: 15, Loss: 1.0871
Epoch: 30, Loss: 0.8105
Epoch: 45, Loss: 0.7743
Epoch: 60, Loss: 0.7632
Epoch: 75, Loss: 0.7585
Epoch: 90, Loss: 0.7564
Epoch: 105, Loss: 0.7564
Epoch: 120, Loss: 0.7553
Epoch: 135, Loss: 0.7547


In [21]:
from sklearn.metrics import accuracy_score

best_acc = {'acc': 0.0}

for p_value in set_of_pq:
    for q_value in set_of_pq:
        model = model_pool[f"model_pq:{p_value}_{q_value}"]
        X_test = model_pool[f"X_test_pq:{p_value}_{q_value}"]
        y_test = model_pool[f"y_test_pq:{p_value}_{q_value}"]

        predict_proba = model.predict_proba(X_test)[:,1]
        y_pred = model.predict(X_test)
        acc = accuracy_score(y_test, y_pred)
        auc = roc_auc_score(y_test, predict_proba)
        if acc > best_acc['acc']:
            best_acc['acc'] = acc
            best_acc['auc'] = auc
            best_acc['p'] = p_value
            best_acc['q'] = q_value

        print(f"--------------Result with p={p_value} and q={q_value}")
        print('Test set AUC: ', auc)
        print('Test set Acc: ', acc)

print(f'---------Best Acc: p={best_acc["p"]} and q={best_acc["q"]}------------------')
print('Test set AUC: ', best_acc['auc'])
print('Test set Acc: ', best_acc['acc'])

--------------Result with p=0.25 and q=0.25
Test set AUC:  0.8935584103255232
Test set Acc:  0.8664556962025316
--------------Result with p=0.25 and q=0.5
Test set AUC:  0.9058290209992683
Test set Acc:  0.8681962025316455
--------------Result with p=0.25 and q=1
Test set AUC:  0.894805742749757
Test set Acc:  0.8719936708860759
--------------Result with p=0.25 and q=2
Test set AUC:  0.8813588629711371
Test set Acc:  0.8685126582278481
--------------Result with p=0.25 and q=4
Test set AUC:  0.8947625
Test set Acc:  0.8637658227848102
--------------Result with p=0.5 and q=0.25
Test set AUC:  0.8994042910191666
Test set Acc:  0.8677215189873417
--------------Result with p=0.5 and q=0.5
Test set AUC:  0.8988507058504432
Test set Acc:  0.8775316455696203
--------------Result with p=0.5 and q=1
Test set AUC:  0.9005782108798329
Test set Acc:  0.8737341772151899
--------------Result with p=0.5 and q=2
Test set AUC:  0.8886148748814041
Test set Acc:  0.8669303797468354
--------------Result wi